In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from functools import reduce

import numpy as np
import DataPreprocessing
import SmartPolynomials

Using TensorFlow backend.


In [2]:
combs = ['1,3','2,3']

In [3]:
##Obtain & process data NPY
me_train, mom_train = DataPreprocessing.npy('NLO/MG_uuxg/me_1000000.npy', 'NLO/MG_uuxg/mom_1000000.npy', [], frac=1)
me_test, mom_test = DataPreprocessing.npy('NLO/MG_uuxg/me_100000.npy', 'NLO/MG_uuxg/mom_100000.npy', [], frac=0.1)

In [4]:
def build_model(components=400):
  model = keras.Sequential([
    keras.layers.Dense(components, activation='relu', input_shape=[12,]), #could try swish??  
    
    keras.layers.Dense(components, activation='relu'),
    keras.layers.Dense(components, activation='relu'),
    keras.layers.Dense(components, activation='relu'),   
    keras.layers.Dense(components, activation='relu'),
      
    keras.layers.Dense(1) 
  ])

  sgd=tf.keras.optimizers.SGD(0.00000000000000005, momentum=0.95, nesterov=True)
  model.compile(loss='mse', 
                optimizer=sgd,
                metrics=['mse'])
  return model

In [5]:
def calc_RMSE(mom_train, mom_test, me_train, me_test, var_name):
    #if var_name == 'Finite':
    #    mom_train = NLOPolynomialFeatures.finitePolynomialFeatures(mandel_str, mom_train, 1000000)
    #    mom_test = NLOPolynomialFeatures.finitePolynomialFeatures(mandel_str, mom_test, 1000000)

    #if var_name == 'Sing':
    #    mom_train = NLOPolynomialFeatures.r1PolynomialFeatures(mandel_str, mom_train,1000000)
    #    mom_test = NLOPolynomialFeatures.r1PolynomialFeatures(mandel_str, mom_test,1000000)
        
    
    mom_train = np.array([np.ndarray.flatten(np.array(element)) for element in mom_train])
    mom_test = np.array([np.ndarray.flatten(np.array(element)) for element in mom_test])

    #if var_name == 'Born' or var_name == 'Doub': 
    #    poly_features = PolynomialFeatures(degree=2)
    #    mom_train = poly_features.fit_transform(mom_train)
    #    mom_test = poly_features.transform(mom_test)
        
    model = build_model()
    scaler = StandardScaler()
    mom_train = scaler.fit_transform(mom_train)
    mom_test = scaler.transform(mom_test)

    model.fit(mom_train,me_train, batch_size=64, epochs=20, shuffle=True)
    pred = model.predict(mom_test,verbose=0) #Prediction on test set

    
    mse_test = mean_squared_error(me_test, pred) #Mean squared error on test set
    mse_train = mean_squared_error(me_train, model.predict(mom_train)) #Mean squared error on train set
    
    print('{} RMSE Test: {}'.format(var_name, np.sqrt(mse_test)))
    print('{} RMSE Train: {}'.format(var_name, np.sqrt(mse_train)))

    perc = np.mean(np.abs(100*np.divide(np.abs(me_test - pred), me_test)))
    print('{} Percentage Error: {}'.format(var_name, perc))

In [6]:
for i, name in enumerate(['Real']):
    i += 1
    div_train = reduce(np.multiply, DataPreprocessing.mandel_creation(['1,3','2,3'], mom_train))
    div_test = reduce(np.multiply, DataPreprocessing.mandel_creation(['1,3','2,3'], mom_test))
    
    temp_train = np.multiply(div_train, me_train[:,i])
    temp_test = np.multiply(div_test, me_test[:,i])
    
    if i == 1 or i == 2:
        div_train = reduce(np.multiply, 1000000-DataPreprocessing.mandel_creation(['1,3','2,3'], mom_train))
        div_test = reduce(np.multiply, 1000000-DataPreprocessing.mandel_creation(['1,3','2,3'], mom_test))

        temp_train = np.multiply(div_train, me_train[:,i])
        temp_test = np.multiply(div_test, me_test[:,i])
        
    calc_RMSE(mom_train, mom_test, temp_train, temp_test, name)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/20
1000000/1000000 [==============================]1000000/1000000 [==============================] - 130s 130us/step - loss: 540786610389068939264.0000 - mean_squared_error: 540786610389068939264.0000

Epoch 2/20
1000000/1000000 [==============================]1000000/1000000 [==============================] - 133s 133us/step - loss: 540786610949109841920.0000 - mean_squared_error: 540786610949109841920.0000

Epoch 3/20
1000000/1000000 [==============================]1000000/1000000 [==============================] - 132s 132us/step - loss: 540786607641616515072.0000 - mean_squared_error: 540786607641616515072.0000

Epoch 4/20
1000000/1000000 [==============================]1000000/1000000 [==============================] - 134s 134us/step - loss: 540786610656762462208.0000 - mean_squared_error: 540786610656762462208.0000

Epoch 5/20
1000000/1000000 [==============================]1000000/1000000 [=======

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1000000/1000000 [==============================]1000000/1000000 [==============================] - 130s 130us/step - loss: 540786610664405860352.0000 - mean_squared_error: 540786610664405860352.0000

Epoch 10/20
1000000/1000000 [==============================]1000000/1000000 [==============================] - 130s 130us/step - loss: 540786609520921411584.0000 - mean_squared_error: 540786609520921411584.0000

Epoch 11/20
  47424/1000000 [>.............................]  47424/1000000 [>.............................] - ETA: 2:03 - loss: 1797632818466862848.0000 - mean_squared_error: 1797632818466862848.0000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1000000/1000000 [==============================]1000000/1000000 [==============================] - 132s 132us/step - loss: 540786609494765797376.0000 - mean_squared_error: 540786609494765797376.0000

Epoch 14/20
 720704/1000000 [====================>.........] 720704/1000000 [====================>.........] - ETA: 36s - loss: 693869632245625913344.0000 - mean_squared_error: 693869632245625913344.0000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1000000/1000000 [==============================]1000000/1000000 [==============================] - 133s 133us/step - loss: 540786614177498071040.0000 - mean_squared_error: 540786614177498071040.0000

Epoch 18/20
 352640/1000000 [=========>....................] 352640/1000000 [=========>....................] - ETA: 1:22 - loss: 79085350618641858560.0000 - mean_squared_error: 79085350618641858560.0000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1000000/1000000 [==============================]1000000/1000000 [==============================] - 131s 131us/step - loss: 540786610913732853760.0000 - mean_squared_error: 540786610913732853760.0000

Real RMSE Test: 369674714.2519558
Real RMSE Train: 23254819063.444725
Real Percentage Error: 100.00122565815643


1.176309223796073e-06
